# Home task: pandas 

## Question 1

- Load the energy data from the file [Energy Indicators.xls](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls).
It is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013.


- It should be put into a DataFrame with the variable name of "energy"


- Make sure to exclude the footer and header information from the datafile.


- The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:<br>
`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`


- Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).


- For all countries which have missing data (e.g. data with `...`) make sure this is reflected as `np.NaN` values.


- Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`


- There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.


- Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). 
It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"


- Make sure to skip the header, and rename the following list of countries:
    - `Korea, Rep.`: `South Korea`,
    - `Iran, Islamic Rep.`: `Iran`,
    - `Hong Kong SAR, China`: `Hong Kong`


- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"


- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

Function "answer_one" should return the resulted DataFrame (20 columns and 15 entries)

## Import all necessary packages

In [34]:
%load_ext blackcellmagic
import pandas as pd
import numpy as np

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


<IPython.core.display.Javascript object>

## Modifying data

### Drop useless stuff, rename column names

In [35]:
# read Excel with all data and drop some useless data
energy_df = (
    pd.read_excel("Energy Indicators.xls", skiprows=16, skipfooter=38)
    .drop(["Unnamed: 0", "Unnamed: 1"], axis=1).drop(0)
)

# rename [Unnamed: 2] -> [Country], [Renewable Electricity Production] -> [% Renewable]
energy_df = energy_df.rename(
    columns={"Unnamed: 2": "Country", "Renewable Electricity Production": "% Renewable"}
)

energy_df.head(5)

,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,321,10,78.66928
2,Albania,102,35,100
3,Algeria,1959,51,0.55101
4,American Samoa,...,...,0.641026
5,Andorra,9,121,88.69565


<IPython.core.display.Javascript object>

### Replace missing data (e.g. data with `...`) with `np.NaN` values.

In [36]:
cols = list(energy_df.columns)[1:]

# First method
# energy_df[cols] = energy_df[cols].apply(lambda x: [item if isinstance(item, (int, float)) else np.NaN for item in x.values])

# Second method
# mask = df[cols].applymap(lambda x: isinstance(x, (int, float)))
# df[cols] = df[cols].where(mask)

# Third method
energy_df[cols] = energy_df[cols].applymap(
    lambda x: x if isinstance(x, (int, float)) else np.NaN
)

print(energy_df.head(5))

energy_df.dtypes

          Country  Energy Supply  Energy Supply per capita  % Renewable
1     Afghanistan          321.0                      10.0    78.669280
2         Albania          102.0                      35.0   100.000000
3         Algeria         1959.0                      51.0     0.551010
4  American Samoa            NaN                       NaN     0.641026
5         Andorra            9.0                     121.0    88.695650


Country                      object
Energy Supply               float64
Energy Supply per capita    float64
% Renewable                 float64
dtype: object

<IPython.core.display.Javascript object>

### Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).

In [51]:
# Converting
energy_df["Energy Supply"] = energy_df["Energy Supply"] * 1_000_000

energy_df.head(5)

<IPython.core.display.Javascript object>

### Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`

In [52]:
rep_dict = {
    "Country": {
        "Republic of Korea": "South Korea",
        "United States of America\d+": "United States",
        "United Kingdom of Great Britain and Northern Ireland\d+": "United Kingdom",
        "China, Hong Kong Special Administrative Region\d+": "Hong Kong",
    }
}

energy_df = energy_df.replace(rep_dict, regex=True)

# check if replace was successful
[
    energy_df.loc[energy_df["Country"] == country]
    for country in ["Hong Kong", "South Korea", "United States", "United Kingdom"]
]

<IPython.core.display.Javascript object>

### There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.

In [53]:
# replace:
# Switzerland17 -> Switzerland
# Bolivia (Plurinational State of) -> Bolivia
energy_df = energy_df.replace(
    {"Country": {r"([A-z]+)\d+": r"\1", r"(\w+) \(.*\)": r"\1"}}, regex=True
)

[
    energy_df.loc[energy_df["Country"] == country]
    for country in ["Switzerland", "Bolivia"]
]

<IPython.core.display.Javascript object>

### Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD).

It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"

Make sure to skip the header, and rename the following list of countries:
- `Korea, Rep.`: `South Korea`,
- `Iran, Islamic Rep.`: `Iran`,
- `Hong Kong SAR, China`: `Hong Kong`

In [54]:
GDP_df = pd.read_excel("world_bank.xls", skiprows=3)

GDP_df.head(2)

<IPython.core.display.Javascript object>

In [55]:
GDP_df = GDP_df.rename(columns={"Country Name": "Country"})

gdp_replace = {
    "Country": {
        "Korea, Rep.": "South Korea",
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong",
    }
}

GDP_df = GDP_df.replace(gdp_replace)

[
    GDP_df.loc[GDP_df["Country"] == country]
    for country in ["South Korea", "Iran", "Hong Kong"]
]

<IPython.core.display.Javascript object>

## Read the last dataset
- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"


- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

In [42]:
ScimEn_df = pd.read_excel('scimagojr country rank 1996-2020.xlsx')

ScimEn_df.head(5)

,Rank,Country,Region,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,Asiatic Region,273437,272374,2336764,1615239,8.55,245
1,2,United States,Northern America,175891,172431,2230544,724472,12.68,363
2,3,India,Asiatic Region,55082,53775,463165,162944,8.41,181
3,4,Japan,Asiatic Region,50523,50065,488062,119930,9.66,193
4,5,United Kingdom,Western Europe,43389,42284,615670,111290,14.19,226


<IPython.core.display.Javascript object>

#### Merge all datasets

In [56]:
merged_df = (
    GDP_df.merge(energy_df, on="Country")
    .merge(ScimEn_df, on="Country")
    .set_index("Country")
)

merge_cols = [
    "Rank",
    "Documents",
    "Citable documents",
    "Citations",
    "Self-citations",
    "Citations per document",
    "H index",
    "Energy Supply",
    "Energy Supply per capita",
    "% Renewable",
    "2006",
    "2007",
    "2008",
    "2009",
    "2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
]

# First method
# merged_df = merged_df.filter(items=merge_cols)

# Second method
merged_df = merged_df[merge_cols].copy()

merged_df = merged_df.sort_values(by=["Rank"])

merged_df.head(5)

<IPython.core.display.Javascript object>

### Create function `answer_one` that returns top-15 countries by Rank

In [59]:
def answer_one(df=merged_df):
    """Returns top-15 countries by rank"""
    return df.loc[df["Rank"] < 16]


answer_one()

<IPython.core.display.Javascript object>

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [60]:
def answer_two():
    """Calculate average GDP for top-15 countries"""
    # Get top-15 values
    Top15 = answer_one()

    # Get only [Year] columns
    avgGDP = Top15.filter(regex="\d+")

    # Calculate the average GDP for each country and sort by descending
    avgGDP = avgGDP.mean(axis=1).sort_values(ascending=False)

    return avgGDP


answer_two()

<IPython.core.display.Javascript object>

### Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [61]:
def answer_three():
    """Calculate how GDP was changed over the 10 years for 6th largst average GDP"""
    # get top-15 values
    Top15 = answer_one()

    # columns list to get
    year_list_to_get = ["2006", "2015"]

    # get 6th largest average GDP
    # first method
    # sixth_country_gdp = Top15.filter(items=year_list_to_get, axis=1).loc['France']

    # second method
    sixth_country_gdp = Top15[year_list_to_get].loc["France"]

    return sixth_country_gdp[1] - sixth_country_gdp[0]


answer_three()

<IPython.core.display.Javascript object>

### Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [63]:
def answer_four():
    """Calculate the ratio of Self-citations to all Citations.
    Find the highest ratio and which country is it.
    """
    # Get top-15 values
    Top15 = answer_one()

    # list of citations to get
    citations_list = ["Citations", "Self-citations"]

    # Get neeeded columns
    # first method
    # df_citations = Top15.filter(items=citations_list)

    # second method
    df_citations = Top15[citations_list].copy()

    # Calculating [Citations Ratio]
    df_citations["Citations Ratio"] = (
        df_citations["Self-citations"] / df_citations["Citations"]
    )
    # Find the highest value
    max_cit_ratio = df_citations["Citations Ratio"].max()

    return (
        max_cit_ratio,
        df_citations[df_citations["Citations Ratio"] == max_cit_ratio].index.values[0],
    )


answer_four()

(0.6912289816173135, 'China')

<IPython.core.display.Javascript object>

### Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [65]:
def answer_five():
    """Calculate population for top-15 countries using energy"""
    # get top-15 values
    Top15 = answer_one()

    # columns to get for energy df
    energy_cols = ["Energy Supply", "Energy Supply per capita"]

    # get needed values
    energy = Top15[energy_cols].copy()

    # calculate population
    energy["Population"] = energy["Energy Supply"] / energy["Energy Supply per capita"]

    # sort values in [Population] column by descending
    energy = energy.sort_values(by="Population", ascending=False)

    # get third population
    third_pop = energy.iloc[2, 2]

    return energy[energy["Population"] == third_pop].index.values[0]


answer_five()

'United States'

<IPython.core.display.Javascript object>

### Question 6
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [67]:
def answer_six():
    """Calculate correlation between citable doc per capita and energy supply per capita"""
    # get top 15 values
    Top15 = answer_one()

    # define columns to get
    taskSix_cols = ["Citable documents", "Energy Supply", "Energy Supply per capita"]

    # get needed info
    taskSix_df = Top15[taskSix_cols].copy()

    # calculate population
    taskSix_df["Population"] = (
        taskSix_df["Energy Supply"] / taskSix_df["Energy Supply per capita"]
    )

    # calculate citable documents per capita
    taskSix_df["Citable documents per capita"] = (
        taskSix_df["Citable documents"] / taskSix_df["Population"]
    )

    return taskSix_df["Citable documents per capita"].corr(
        taskSix_df["Energy Supply per capita"]
    )


answer_six()

0.7434709127726777

<IPython.core.display.Javascript object>

### Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [68]:
def answer_seven():
    """Create DF with specific info like: size, sum, mean, std. Index have to be the Continent name

    Returns:
        [DataFrame]: returns DataFrame with all info
    """
    # get top-15 values
    Top15 = answer_one()

    # define columns to get
    taskSeven_cols = ["Energy Supply", "Energy Supply per capita"]

    # get needed info
    taskSeven_df = Top15[taskSeven_cols].copy()

    # calculate population
    taskSeven_df["Population"] = (
        taskSeven_df["Energy Supply"] / taskSeven_df["Energy Supply per capita"]
    )

    # create dict {[country]: [continent]}
    ContinentDict = {
        "China": "Asia",
        "United States": "North America",
        "Japan": "Asia",
        "United Kingdom": "Europe",
        "Russian Federation": "Europe",
        "Canada": "North America",
        "Germany": "Europe",
        "India": "Asia",
        "France": "Europe",
        "South Korea": "Asia",
        "Italy": "Europe",
        "Spain": "Europe",
        "Iran": "Asia",
        "Australia": "Australia",
        "Brazil": "South America",
    }

    # add [Continent] column
    taskSeven_df["Continent"] = [ContinentDict[x] for x in taskSeven_df.index.tolist()]

    # get continents list
    continent_indexes = taskSeven_df.groupby("Continent").count().index.tolist()

    # find how much countries we have per continent
    countries_per_continent = (
        taskSeven_df["Continent"].value_counts().sort_index().tolist()
    )
    # print(taskSeven_df['Continent'].value_counts().sort_index().tolist())

    # sum population
    sum_population = taskSeven_df.groupby(["Continent"]).sum()["Population"].tolist()
    # print(taskSeven_df.groupby(['Continent']).sum()['Population'].tolist())

    # mean population
    mean_population = taskSeven_df.groupby(["Continent"]).mean()["Population"].tolist()
    # print(taskSeven_df.groupby(['Continent']).mean()['Population'].tolist())

    # std population
    std_population = taskSeven_df.groupby(["Continent"]).std()["Population"].tolist()
    # print(taskSeven_df.groupby(['Continent']).std()['Population'])

    # create result df
    result_df = pd.DataFrame(
        {
            "size": countries_per_continent,
            "sum": sum_population,
            "mean": mean_population,
            "std": std_population,
        },
        index=continent_indexes,
        columns=["size", "sum", "mean", "std"],
    )

    return result_df


answer_seven()

<IPython.core.display.Javascript object>